In [1]:
ACCOUNT_ROMANAME = "kano"

In [2]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.options import Options
from dotenv import load_dotenv

load_dotenv()
def login(USER_NAME, PASSWORD, OAUTH):
    # usernameのinput要素を指定
    username = driver.find_element(By.ID,'username')
    time.sleep(1)
    
    # フィールドの内容をクリア
    username.clear()
    # usernameのinput要素にログインid(email)を入力
    username.send_keys(USER_NAME)
    time.sleep(1)

    # passwordのinput要素を指定
    password = driver.find_element(By.ID,'password')
    time.sleep(1)

    # passwordのinput要素にパスワードを入力
    password.send_keys(PASSWORD)
    time.sleep(1)

    # ログインボタンを指定
    log_in_button = driver.find_element(By.XPATH,'//*[@type="submit"]')
    time.sleep(1)

    # ログインボタンをクリック
    log_in_button.click()
    time.sleep(1)

chrome_options = Options()
# その他のオプション設定が必要な場合、ここに追加...
# 高速化したいときはheadlessモードで起動
# chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

driver.maximize_window()
driver.get('https://www.linkedin.com/login/ja?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')


load_dotenv(f"config./.{ACCOUNT_ROMANAME}.env")   
USER_NAME = os.environ.get("USER_NAME")
PASSWORD = os.environ.get("PASSWORD")
OAUTH = ""
login(USER_NAME, PASSWORD, OAUTH)
time.sleep(5)

time.sleep(5)
driver.execute_script("window.open('https://www.linkedin.com/sales/home', '_blank');")
driver.switch_to.window(driver.window_handles[-1])
time.sleep(5)

try:
    driver.get('https://www.linkedin.com/sales/search/people?viewAllFilters=true')
except:
    print("手動でフィルターをかけてください")




### 検索フィルタを設置後、スクレイピング開始
- 基本的には"Role"-->"Seniority"で役職を選択してください
- その他のフィルタは必要に応じて選択してください


In [3]:
import csv
from datetime import datetime
import os
import pytz

# 日本時間の設定
JST = pytz.timezone('Asia/Tokyo')

# CSVファイル名を現在時刻で生成
timestamp = datetime.now(JST).strftime("%Y%m%d_%H%M%S")
csv_folder = 'linkedin_data'
os.makedirs(csv_folder, exist_ok=True)
csv_filename = os.path.join(csv_folder, f'linkedin_data_{timestamp}.csv')

# 既存のデータを読み込む
existing_data = []
if os.path.exists(csv_filename):
    with open(csv_filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        existing_data = list(reader)
        if existing_data:
            last_entry = existing_data[-1]  # 最後の行を取得
        else:
            last_entry = None
else:
    # 新規ファイルの場合、ヘッダーを書き込む
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["名前", "プロフィールURL", "取得日"])
    last_entry = None

page_num = 1
today = datetime.now(JST).strftime("%Y%m%d")
while True:
    print(f"ページ {page_num} の処理を開始")
    
    try:
        # 毎回要素を再取得
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH, 
                "//div[@id='search-results-container' and contains(@class, 'overflow-x-hidden') and contains(@class, 'overflow-y-auto') and contains(@class, 'p4')]"
            ))
        )
        
        # 上から下まで徐々にスクロール
        last_height = driver.execute_script("return arguments[0].scrollHeight", element)
        current_position = 0
        while current_position < last_height:
            # 500pxずつスクロール
            current_position += 500
            driver.execute_script("arguments[0].scrollTo(0, arguments[1]);", element, current_position)
            time.sleep(1)
        
        # データ取得
        person_names = driver.find_elements(By.XPATH, "//span[@data-anonymize='person-name']")
        person_links = driver.find_elements(By.XPATH, "//span[@data-anonymize='person-name']/ancestor::a")
        
        # データを整形
        person_data = []
        for name, link in zip(person_names, person_links):
            current_entry = [name.text, link.get_attribute('href'), today]
            # 最後のエントリーの後のデータのみを追加
            if last_entry:
                if current_entry == last_entry:
                    last_entry = None  # 最後のエントリーを見つけた後は全て追加
                    continue
                if last_entry is not None:
                    continue
            person_data.append(current_entry)
        
        # 新しいデータがある場合のみCSVに追記
        if person_data:
            with open(csv_filename, 'a', newline='', encoding='utf-8') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerows(person_data)
            print(f"ページ {page_num} のデータを保存しました（{len(person_data)}件）")
        
        # 次ページボタンを探して遷移
        next_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".artdeco-pagination__button--next"))
        )
        
        # ボタンが無効化されているかチェック
        if 'disabled' in next_button.get_attribute('class'):
            print("最終ページに到達しました")
            break
            
        # 次ページへ遷移
        next_button.click()
        time.sleep(3)  # ページ遷移の待機
        page_num += 1
        
    except TimeoutException:
        print("要素が見つかりません。処理を終了します")
        break
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        break

print(f"スクレイピングが完了しました。データは {csv_filename} に保存されています。")

ページ 1 の処理を開始
ページ 1 のデータを保存しました（25件）
ページ 2 の処理を開始


KeyboardInterrupt: 